In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import glob
# import face_recognition
import numpy as np
import joblib
import uuid
import seaborn as sns

In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model, Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten
import imutils

In [3]:
SIZE = 240

In [7]:
# Training data and labels into respective lists
train_images = []
train_labels = []
prev=0

for directory_path in glob.glob("/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        # print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
    print(len(train_images)-prev)
    prev=len(train_images)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Daniel Radcliffe
50
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Alia Bhatt
50
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Aishwarya Rai
49
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Darshan Raval
46
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Gal Gadot
48
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Honey Singh
45
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Emma Watson
50
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Hrithik Roshan
47
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Sharukh Khan
47
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Kaley Cuoco
49
/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Mr. Bean
46
/co

In [8]:
len(train_labels)

571

In [9]:
#Encode labels from text to integers.
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
train_labels_encoded = encoder.transform(train_labels)

In [10]:
x_train, y_train = train_images, train_labels_encoded
# Normalize pixel values to between 0 and 1
x_train = x_train / 255.0

In [11]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=45)

In [14]:
# Loading model
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

58889256/58889256 [==============================] - 2s 0us/step


In [15]:
for layer in VGG_model.layers:
	layer.trainable = False
    
x=Flatten()(VGG_model.output)
VGG_model = Model(inputs=VGG_model.input, outputs=x)

# VGG_model.summary()

In [16]:
# feature_extractor=VGG_model.predict(x_train)
features=VGG_model.predict(X_train)

13/13 [==============================] - 13s 284ms/step


In [22]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [23]:
RF_model.fit(features, Y_train)

RandomForestClassifier(random_state=42)

In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

In [ ]:
protoPath = "D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\face_detector\\deploy.prototxt"
weightsPath = "D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel"

faceNet = cv2.dnn.readNet(protoPath, weightsPath)

In [ ]:
maskNet = load_model('D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\mask_detector.model')

In [ ]:
VGG_model.save('featuresVGG.h5')
joblib.dump(RF_model, "RFModel.joblib")

['RFModel.joblib']

In [ ]:
VGG_model = load_model("D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\featuresVGG.h5")
RF_model = joblib.load("D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\RFModel.joblib")

In [24]:
def predict(img):
    if img is None:
        return 'Unknown'1
    img = cv2.resize(img,(SIZE,SIZE))
    input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=VGG_model.predict(input_img)
    prediction_RF = RF_model.predict(input_img_feature)[0]
    # prediction_RF = RF_model.predict_proba(input_img_feature)
    prediction_RF = encoder.inverse_transform([prediction_RF])  #Reverse the label encoder to original name
    return str(prediction_RF)
    # return prediction_RF

In [26]:
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Alia Bhatt/6889155f-5d48-11ed-9c5b-90324b084670.jpg')
# max = np.max(predict(img))
print(predict(img))
# max

1/1 [==============================] - 0s 27ms/step
['/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Sharukh Khan']


In [ ]:
cap = cv2.VideoCapture(0)

while True:
	_ , frame = cap.read()

	# detect faces with mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations
	for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY-100), (0,0,0), 2)
		img = frame[startY:endY-100,startX:endX]
		
		cv2.imshow("cut",img)
		name = predict(img)
		cv2.putText(frame, name, (endX-20, endY+10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
  
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	if key == ord("q"):
		break

cap.release()
cv2.destroyAllWindows()